In [29]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from collections import defaultdict
train_data = pd.read_csv('./data/book_ratings_train.csv')
test_data = pd.read_csv('./data/book_ratings_test.csv')
user_data = pd.read_csv('./data/users.csv')
all_user_book_pair = pd.concat([train_data, test_data])[['User-ID', 'ISBN']]



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Remove the CWD from sys.path while we load stuff.


In [30]:
print(train_data.shape)
print(test_data.shape)
print(len(train_data['User-ID'].unique()))
print(len(train_data['ISBN'].unique()))
print(len(all_user_book_pair['User-ID'].unique()))
print(len(all_user_book_pair['ISBN'].unique()))


(260202, 3)
(173469, 2)
57758
128932
77805
185973


In [31]:
user2id = {}
book2id = {}
for i, userID in enumerate(all_user_book_pair['User-ID'].unique()):
    user2id[userID] = i
for j, bookID in enumerate(all_user_book_pair['ISBN'].unique()):
    book2id[bookID] = j
train_user = np.array([user2id[i] for i in train_data['User-ID']])
train_book = np.array([book2id[j] for j in train_data['ISBN']])
trainRating = np.array(train_data['Book-Rating'])
mean = np.mean(trainRating)
std = np.std(trainRating)

#train_rating_normalized = (train_rating - mean) / std
print(mean, std)
print(np.mean(np.array(user_data['Age'])))

7.601225201958479 1.844145990351842
nan


In [40]:
user_data = np.array(user_data)
age=[]
for user in user_data:
    if not np.isnan(user[2]):
        age.append(user[2])
print(sum(age)/len(age))

34.75143370454978


In [42]:
train_data = np.array(train_data)
user_data = np.array(user_data)
print(user_data[0])
userRating = defaultdict(list)
bookRating = defaultdict(list)
userRegion = defaultdict(int)
userAge = defaultdict(int)
for data in train_data:
    userRating[data[0]].append(data[2])
    bookRating[data[1]].append(data[2])
for user in user_data:
    if 'usa' in user[1]:
        userRegion[user[0]] = 1
    else:
        userRegion[user[0]] = 0
    if not np.isnan(user[2]):
        userAge[user[0]] = user[2]
    else:
        userAge[user[0]] = 34.75

trainingFeatures = []
for data in train_data:
    userRatings = userRating[data[0]]
    bookRatings = bookRating[data[1]]
    age = userAge[data[0]]
    region = userRegion[data[0]]
    userMean = np.mean(userRatings)
    userStd = np.std(userRatings)
    userVar = np.var(userRatings)
    userRatingNum = len(userRatings)
    userMin = np.min(userRatings)
    bookMean = np.mean(bookRatings)
    bookStd = np.std(bookRatings)
    bookVar = np.var(bookRatings)
    bookRatingNum = len(bookRatings)
    bookMin = np.min(bookRatings)
    trainingFeatures.append([age,region,userMean,userStd,userVar,userRatingNum,bookMean,bookStd,bookVar,bookRatingNum,userMin,bookMin])
trainingFeatures = np.array(trainingFeatures)
#trainRating = np.array(train_data[2])
print(trainingFeatures.shape)
print(trainingFeatures[0])



['7c9fa136d4' 'nyc, new york, usa' nan]
(260202, 12)
[ 47.           0.           7.33227848   1.17880915   1.38959101
 632.           8.           0.           0.           1.
   3.           8.        ]


In [43]:
gb = GradientBoostingRegressor(n_estimators=200,max_depth=4)
gb.fit(trainingFeatures, trainRating)
print(gb.feature_importances_)

[0.01102189 0.00086321 0.15654567 0.07236898 0.07210827 0.0364212
 0.18038701 0.09923147 0.10877483 0.05391155 0.10532305 0.10304288]


In [ ]:
forest = RandomForestRegressor(n_estimators = 20,criterion='mae',max_depth=2)
forest.fit(trainingFeatures, trainRating)
print(forest.feature_importances_)


In [47]:
test_data = np.array(test_data)
testingFeatures = []
for data in test_data:
    userRatings = userRating[data[0]]
    bookRatings = bookRating[data[1]]
    age = userAge[data[0]]
    region = userRegion[data[0]]
    if userRatings == []:
        userMean = mean
        userStd = std
        userVar = std**2
        userRatingNum = 1
        userMin = mean
    else:
        userMean = np.mean(userRatings)
        userStd = np.std(userRatings)
        userVar = np.var(userRatings)
        userRatingNum = len(userRatings)
        userMin = np.min(userRatings)
    if bookRatings == []:
        bookMean = mean
        bookStd = std
        bookVar = std**2
        bookRatingNum = 1
        bookMin = mean
    else:
        bookMean = np.mean(bookRatings)
        bookStd = np.std(bookRatings)
        bookVar = np.var(bookRatings)
        bookRatingNum = len(bookRatings)
        bookMin = np.min(bookRatings)
    testingFeatures.append([age,region,userMean,userStd,userVar,userRatingNum,bookMean,bookStd,bookVar,bookRatingNum,userMin,bookMin])
testingFeatures = np.array(testingFeatures)

print(testingFeatures.shape)

(173469, 12)


In [48]:
result = gb.predict(testingFeatures)

In [49]:
print(len(result))
print(result[0:5])
print(np.around(result[0:5]))

173469
[7.68962319 7.21664223 8.49809213 6.38170999 8.85024112]
[8. 7. 8. 6. 9.]


In [50]:
with open('ans.csv', 'w') as f:
    for i in range(len(result)):
        f.write('%d\n' %(result[i]))
        
with open('ansround.csv', 'w') as f:
    for i in range(len(result)):
        f.write('%d\n' %(int(np.round(result[i]))))